# Volleyball Data handling

This notebook will be used handling data regarding volleyball courts, taken from the datasets supplied by Active Places Power, Sport England (https://www.activeplacespower.com/pages/downloads#download)  

In [1]:
# import pandas

import pandas as pd

# Dataset Information

## sportshalls.csv

This dataset stores data about various sports halls around England, informing about the different courts available, the size of halls, etc.

## sites.csv

This dataset stores information about the sites themselves, as the sports halls do not store the name and information about the location itself, which I will have to use the siteID from sportshalls.csv to link to the siteID in sites.csv

In [2]:
# read sports hall csv
sports_halls = pd.read_csv('activeplacescsvs/sportshalls.csv', low_memory=False)

# read sites information csv
sites = pd.read_csv('activeplacescsvs/sites.csv')

In [3]:
sports_halls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14372 entries, 0 to 14371
Data columns (total 65 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Badminton Courts                               14371 non-null  float64
 1   Basketball Courts                              14371 non-null  float64
 2   Clearance exists - Ball / shuttlecock          14371 non-null  float64
 3   Cricket Nets                                   14371 non-null  float64
 4   Dimensions Estimate                            14372 non-null  int64  
 5   Facility ID                                    14372 non-null  int64  
 6   Five-A-Side Pitches                            14371 non-null  float64
 7   Floor Matting                                  14371 non-null  float64
 8   Futsal Courts                                  6264 non-null   float64
 9   Gymnastics / Trampoline Use                    143

In [4]:
sites.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41981 entries, 0 to 41980
Data columns (total 79 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Active Partnership Code                        41943 non-null  object 
 1   Active Partnership Name                        41981 non-null  object 
 2   Cycle Hire?                                    41981 non-null  int64  
 3   Building Name                                  11487 non-null  object 
 4   Building Number                                3254 non-null   float64
 5   Car Park Capacity                              41980 non-null  float64
 6   Car Park Exists?                               41981 non-null  int64  
 7   Changing Place Toilets                         41981 non-null  int64  
 8   Closure Reason                                 4970 non-null   float64
 9   Core City Name                                 265

# Data cleaning

First step was to only select the columns that I believe to be of importance. 

I will require the __SiteID__ and __FacilityID__ to be able to link the information about the sports halls with the names of the facilities and locations of them.

I will need to gather various information about the sports halls in regards to volleyball, such as __number of vb courts__, the __size__ of the sports halls, if they __have changing rooms__, and other information that may be useful for people.

when visualising the data, there are a number of sports halls which do not have any volleyball courts available which I will have to remove from the dataset. 



## Sports halls data clean-up

### Variable documentation

- __sports_halls_volley__: specific columns selected of importance
- __vb_courts_count__: count of number of vb courts occurrence
- __filtered_vb_courts__: filtered for only records with >=1 vb courts

In [5]:
# Selecting only specific columns about sports halls
sports_halls_volley = sports_halls[['Site ID', 'Facility ID', 'Volleyball Courts', 'Length', 'Width', 'Has changing rooms?', 'Facility Type', 'Facility Subtype', 'Latitude', 'Longitude', 'Accessibility Type Group (Text)']]

# Renaming certain columns that have unnecessarily long names
sports_halls_volley.rename({'Has changing rooms?': 'Changing Rooms', 'Accessibility Type Group (Text)': 'Access Type'}, axis=1, inplace=True)

sports_halls_volley.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14372 entries, 0 to 14371
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Site ID            14372 non-null  int64  
 1   Facility ID        14372 non-null  int64  
 2   Volleyball Courts  14372 non-null  int64  
 3   Length             14371 non-null  float64
 4   Width              14372 non-null  float64
 5   Changing Rooms     14372 non-null  int64  
 6   Facility Type      14372 non-null  int64  
 7   Facility Subtype   14372 non-null  int64  
 8   Latitude           14372 non-null  float64
 9   Longitude          14372 non-null  float64
 10  Access Type        14372 non-null  object 
dtypes: float64(4), int64(6), object(1)
memory usage: 1.2+ MB


C:\Users\kacpe\AppData\Local\Temp\ipykernel_68196\2485649503.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sports_halls_volley.rename({'Has changing rooms?': 'Changing Rooms', 'Accessibility Type Group (Text)': 'Access Type'}, axis=1, inplace=True)


In [6]:
sports_halls_volley.head()

,Site ID,Facility ID,Volleyball Courts,Length,Width,Changing Rooms,Facility Type,Facility Subtype,Latitude,Longitude,Access Type
0,1000004,2000007,0,34.5,20.0,1,6,6001,50.85914,-0.089625,Public Access
1,1000009,2000019,1,40.0,30.5,1,6,6001,53.26373,-2.891404,Public Access
2,1000010,2000022,0,18.0,17.0,1,6,6002,50.42324,-4.214078,Public Access
3,1000018,2000034,0,34.5,20.0,1,6,6001,51.58628,0.462723,Public Access
4,1000019,2000036,1,33.0,18.0,1,6,6001,52.55358,-1.897004,Public Access


In [7]:
# Checking occurrence number of volleyball courts for each sports hall

vb_courts_count = sports_halls_volley['Volleyball Courts'].value_counts()

print(vb_courts_count.sort_index())

Volleyball Courts
0    9634
1    4403
2     281
3      40
4      13
8       1
Name: count, dtype: int64


In [8]:
# Remove all records with 0 vb courts
filtered_vb_courts = sports_halls_volley[sports_halls_volley['Volleyball Courts'] > 0]

print(filtered_vb_courts['Facility Type'].value_counts().sort_index())

Facility Type
6    4738
Name: count, dtype: int64


In [9]:
print(filtered_vb_courts['Volleyball Courts'].value_counts().sort_index())

Volleyball Courts
1    4403
2     281
3      40
4      13
8       1
Name: count, dtype: int64


In [10]:
# Total of 0 sports halls with no volleyball courts
mask = filtered_vb_courts['Volleyball Courts'] == 0

print(mask.sum())

0


In [11]:
# Final result
filtered_vb_courts.head()

,Site ID,Facility ID,Volleyball Courts,Length,Width,Changing Rooms,Facility Type,Facility Subtype,Latitude,Longitude,Access Type
1,1000009,2000019,1,40.0,30.5,1,6,6001,53.26373,-2.891404,Public Access
4,1000019,2000036,1,33.0,18.0,1,6,6001,52.55358,-1.897004,Public Access
11,1000056,2000107,1,34.5,20.0,1,6,6001,53.55635,-2.233766,Public Access
13,1000070,2000129,1,27.0,18.0,1,6,6001,51.39739,-1.298654,Public Access
21,1000097,2000180,1,30.0,24.0,1,6,6001,51.51729,-0.130378,Public Access


In [12]:
nan_vb = filtered_vb_courts.isna().sum()
print(nan_vb)

Site ID              0
Facility ID          0
Volleyball Courts    0
Length               0
Width                0
Changing Rooms       0
Facility Type        0
Facility Subtype     0
Latitude             0
Longitude            0
Access Type          0
dtype: int64


I now have the __filtered_vb_courts__ variable which is the cleaned version of the __sports_halls__ dataset

## Sites data clean-up

In [13]:
# Selecting only specific columns about sports halls
sites_updated = sites[['Site ID', 'Site Name', 'Car Park Capacity', 'Cycle Park?', 'Local Authority Name', 'Postcode', 'Town', 'Region Name', 'lat', 'long']]

sites_updated.head()

,Site ID,Site Name,Car Park Capacity,Cycle Park?,Local Authority Name,Postcode,Town,Region Name,lat,long
0,30010710,TEN ARCE FIELD,0.0,0,West Suffolk,IP32 7DA,BURY ST. EDMUNDS,East of England,52.24416,0.735483
1,30003564,EASTCOMBE SCOUT HUT PITCH,30.0,0,Stroud,GL6 7DY,STROUD,South West,51.73487,-2.157465
2,1005515,MOTCOMBE PARK SPORTS CLUB,400.0,1,Dorset,SP7 9QA,SHAFTESBURY,South West,51.01973,-2.216240
3,1008868,YORK ST JOHN UNIVERSITY (YSJACTIVE GYM),100.0,1,York,YO31 7EX,YORK,Yorkshire and The Humber,53.96486,-1.079552
4,30007658,BELMONT MILL HILL PREPARATORY SCHOOL,30.0,1,Barnet,NW7 4ED,LONDON,London,51.62357,-0.234825


In [14]:
site_disability_info = sites[['Site ID', 'Disability: Activity Area', 'Disability: Changing Facilities', 'Disability: Doorways', 'Disability: Emergency Exits', 'Disability: Finding and reaching the entrance', 'Disability: Parking', 'Disability: Reception Area', 'Disability: Social Areas', 'Disability: Spectator Areas', 'Disability: Toilets']]

In [15]:
# Find if there are any NaN values
nan_sites = sites_updated.isna().sum()
print(nan_sites)

Site ID                 0
Site Name               0
Car Park Capacity       1
Cycle Park?             0
Local Authority Name    0
Postcode                0
Town                    0
Region Name             0
lat                     0
long                    0
dtype: int64


In [16]:
# Find row with NaN value 
empty = sites_updated.isna().any(axis=1)
empty_rows = sites_updated[empty]
print(empty_rows)

        Site ID                  Site Name  Car Park Capacity  Cycle Park?  \
18801  30009987  PRINCE ALBERT HIGH SCHOOL                NaN            1   

      Local Authority Name Postcode        Town    Region Name       lat  \
18801           Birmingham  B42 2TU  BIRMINGHAM  West Midlands  52.52111   

           long  
18801 -1.894346  


In [17]:
# Since only NaN value is in car park capacity
# Assume car park has no space so fill with 0
cleaned_sites = sites_updated.fillna(0)
print(cleaned_sites.isna().sum())


Site ID                 0
Site Name               0
Car Park Capacity       0
Cycle Park?             0
Local Authority Name    0
Postcode                0
Town                    0
Region Name             0
lat                     0
long                    0
dtype: int64
